# Test

In [1]:
import os
import sys
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import safe_crosstab

### Load test data

In [5]:
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.head()

,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
0,4Children,R,1.0,2011,2880902.0,9603182.0,91404.0,310947.0,12886435.0,12127472.0,...,NaN,NaN,0.072636,0.135971,0.767809,0.094602,0.177090,0.534203,5.646843,1.0
1,4Children,R,1.0,2014,6810520.0,18768904.0,58002.0,401879.0,26039304.0,25493796.0,...,NaN,NaN,0.057641,0.089150,1.001396,0.057560,0.089026,0.646561,11.232729,1.0
2,4Children,R,1.0,2015,7199403.0,21638036.0,132191.0,512654.0,29482284.0,32290108.0,...,NaN,NaN,-0.049619,-0.079828,-0.620210,0.080004,0.128711,0.621583,7.769365,1.0
3,4Children,R,1.0,2013,5573013.0,15194731.0,228844.0,267156.0,21263744.0,20989048.0,...,NaN,NaN,0.045740,0.068251,1.008259,0.045365,0.067692,0.670166,14.772749,1.0
4,4Children,R,1.0,2010,2056816.0,7335103.0,110256.0,424628.0,9926803.0,9769816.0,...,NaN,NaN,0.057696,0.122532,0.567539,0.101660,0.215901,0.470862,4.631749,1.0


### Pandas crosstab

In [5]:
table = pd.crosstab(df["year"], df["grant_type"])
table

grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,8
2011,15,59,71,8
2012,15,59,71,8
2013,15,59,71,8
2014,15,59,71,8
2015,15,59,71,8


### Safe crosstab

In [6]:
safe_table = safe_crosstab(df["year"], df["grant_type"])
safe_table

DEBUG:acro:crosstab()
DEBUG:acro:args: {'output_template_file': 'ACRO output template v01b.xlsm', 'safe_SDC_set': 'Default', 'safe_tests': 'threshold nk pratio maxmin', 'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'safe_SDC_variations': 'CIS ESS', 'safe_thresholdCIS': 60, 'safe_nk_nCIS': 5, 'safe_nk_kCIS': 0.5, 'safe_testsCIS': 'threshold nk pratio', 'safe_thresholdESS': 15, 'safe_dof_thresholdESS': 10, 'safe_nk_nESS': 2, 'safe_nk_kESS': 0.9, 'safe_pratio_pESS': 0.15, 'safe_testsESS': 'nk pratio'}
DEBUG:acro:suppressing 6 cells where value < threshold


grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,n/a
2011,15,59,71,n/a
2012,15,59,71,n/a
2013,15,59,71,n/a
2014,15,59,71,n/a
2015,15,59,71,n/a


## Identify a sub-group to edit for checking dominance and p% ratio

- From previous analysis there were only 15 charities getting grants of type G each year so let's target that in 2010
- Starting by creating a new dataset to see what is in that cell
- and sorting it in descending order 

In [95]:
my_field = "inc_grants"
g_in_2010 = df[(df["grant_type"] == "G") & (df["year"] == 2010)].sort_values(
    my_field, ascending=False
)

print(f"\nTotal  income from {my_field} in this group is {g_in_2010[my_field].sum()}\n")
g_in_2010.head(15)


Total  income from inc_grants in this group is 138906688.0



,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
147,British Red Cross,G,118.0,2010,24800000.0,57600000.0,119900000.0,600000.0,213800000.0,230900000.0,...,0.375000,1.651899,0.728718,2.238506,9.860760,0.073901,0.227011,0.325538,4.405063,1.0
174,Cancer Research UK,G,121.0,2010,82835000.0,46628000.0,362756000.0,5538000.0,514945984.0,466968000.0,...,1.474797,1.084388,0.353051,1.686084,1.239742,0.284777,1.360028,0.209391,0.735279,3.0
831,The Prince's Trust,G,196.0,2010,4641000.0,14175000.0,17440000.0,429000.0,34283000.0,35878000.0,...,0.086058,0.036976,0.975819,2.327397,1.000000,0.975819,2.327397,0.419275,0.429665,5.0
840,The Royal British Legion,G,197.0,2010,42876000.0,8670000.0,66450000.0,4842000.0,115235000.0,114486000.0,...,0.189826,0.019408,2.534525,9.845806,1.006666,2.517742,9.780609,0.257422,0.102243,3.0
916,age UK,G,203.0,2010,103998000.0,8624000.0,39670000.0,899000.0,160664992.0,159896000.0,...,1.881926,0.706569,0.078468,1.088124,0.408536,0.192070,2.663473,0.072113,0.375450,4.0
537,Macmillian Cancer Support,G,159.0,2010,1076000.0,3066000.0,127393000.0,2070000.0,133605000.0,132949000.0,...,2.821259,1.352288,0.529209,2.077237,0.995663,0.531515,2.086286,0.254766,0.479321,1.0
142,British Heart Foundation,G,117.0,2010,116377000.0,93000.0,91036000.0,6256000.0,213762000.0,203104000.0,...,1.010922,0.935604,0.242054,1.080502,1.000000,0.242054,1.080502,0.224020,0.925496,3.0
625,Oakley regeneration,G,35.0,2010,97227.0,47690.0,0.0,0.0,146174.0,130632.0,...,NaN,6.862763,0.119987,NaN,1.000000,0.119987,NaN,0.000000,0.000000,2.0
445,Help for Heroes,G,151.0,2010,8825000.0,3000.0,36439000.0,459000.0,45723000.0,42945000.0,...,33.987015,9.142358,0.167749,12.451299,3.349345,0.050084,3.717532,0.013472,0.268996,2.0
225,Children in need,G,126.0,2010,0.0,0.0,42611000.0,1593000.0,44204000.0,46974000.0,...,16.204733,1.373527,0.710479,11.797896,1.000000,0.710479,11.797896,0.060221,0.084761,1.0


### some stats so we know the magnitude of the change we need to make

In [96]:
num1 = g_in_2010.iloc[0]
num2 = g_in_2010.iloc[1]
total = g_in_2010[my_field].sum()
rest_total = total - num1[my_field] - num2[my_field]

print(
    f"The total of grants is  {total}\n"
    f" and the two highest values are:\n"
    f"         {num1['charity']:20s} with {num1[my_field]}\n"
    f"     and {num2['charity']:20s} with {num2[my_field]}\n"
    f"so the rest make up {rest_total}\n"
    f" which is {rest_total*100/num1[my_field]}%"
    f" of the value for {num1['charity']:20s}"
)

p = 10
diff = (100 / p) * rest_total - num1[my_field]
print(
    f"So for the cell to fail the p% rule with p={p} "
    f'we need {num1["charity"]} total \nto be more than (100/{p})*{rest_total}\n'
    f"which we can achieve by adding {diff}"
)

The total of grants is  138906688.0
 and the two highest values are:
         British Red Cross    with 57600000.0
     and Cancer Research UK   with 46628000.0
so the rest make up 34678688.0
 which is 60.20605555555556% of the value for British Red Cross   
So for the cell to fail the p% rule with p=10 we need British Red Cross total 
to be more than (100/10)*34678688.0
which we can achieve by adding 289186880.0


### So lets add that amount to the British Red cross totals in a new version of the table

In [120]:
df2 = df.copy(deep=True)

to_add = 290000000

to_change = []
for row in range(len(df2)):
    if (
        (df2.loc[row]["grant_type"] == "G")
        & (df2.loc[row]["year"] == 2010)
        & (df2.loc[row]["charity"] == "British Red Cross")
    ):
        to_change.append(row)

if len(to_change) != 1:
    print(f"error identified {len(to_change)} matching rows")
else:
    row = to_change[0]
    for col in [my_field, "inc_total"]:
        oldval = df2.at[row, col]
        df2.at[row, col] = oldval + to_add
df2.describe()

,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,grants_given,balance,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
count,918.000000,918.000000,815.0,815.0,811.0,804.00,815.0,815.0,7.160000e+02,8.130000e+02,...,649.000000,711.000000,813.000000,746.000000,807.000000,815.000000,748.000000,810.000000,804.000000,918.000000
mean,110.660133,2012.500000,11784089.0,6353625.5,15988774.0,445775.25,35098612.0,32809674.0,7.982982e+06,3.502526e+07,...,0.946936,0.144413,1.862605,12.974613,0.786334,1.864529,12.958950,0.443249,2.339051,2.450980
std,64.716599,1.708756,42843424.0,26805158.0,47023004.0,1198416.50,86426816.0,80507224.0,9.384350e+07,1.189963e+08,...,5.740510,3.466396,4.881787,87.064911,10.627007,4.868544,86.958290,0.495077,17.996527,2.081044
min,1.000000,2010.000000,-3459.0,0.0,0.0,-556000.00,5557.0,8197.0,0.000000e+00,-3.060000e+07,...,0.000000,-56.687500,-0.772064,-3.059903,-289.639709,-0.097056,-0.417188,0.000000,-39.470589,1.000000
25%,39.000000,2011.000000,11212.0,13067.0,16257.5,134.50,494715.0,458550.0,0.000000e+00,2.015130e+05,...,0.000000,0.000000,0.271728,0.571918,1.000000,0.276006,0.613534,0.247473,0.174587,1.000000
50%,127.000000,2012.500000,205165.0,293041.0,235141.0,7207.00,948875.0,924258.0,0.000000e+00,1.158000e+06,...,0.000000,0.000000,0.547965,1.258261,1.000000,0.546640,1.257283,0.420163,0.660209,2.000000
75%,165.000000,2014.000000,4212500.0,1519054.5,6113000.0,180896.50,13805875.0,13880316.0,0.000000e+00,1.282200e+07,...,0.000000,0.000000,1.293503,3.399237,1.000000,1.290790,3.305527,0.612311,1.461616,3.000000
max,203.000000,2015.000000,421265984.0,347600000.0,475668000.0,9200000.00,680800000.0,602400000.0,2.456000e+09,1.149684e+09,...,68.274368,36.623749,38.465309,1098.345825,37.148750,38.465309,1098.345825,11.449343,464.588135,11.000000


### print the cross tab with values='inc_grant'

In [121]:
grant_table = pd.crosstab(
    df2["year"], df2["grant_type"], values=df2[my_field], aggfunc=sum
)
grant_table

grant_type,G,N,R,R/G
year,,,,
2010,428906688.0,0.0,504137056.0,46544000.0
2011,127533696.0,7192804.0,532464736.0,128380000.0
2012,171878704.0,7779685.0,480105472.0,134480000.0
2013,203357200.0,8728330.0,511361440.0,134125000.0
2014,206222208.0,7858697.0,554594176.0,142766000.0
2015,133601200.0,8501187.0,551457280.0,146228992.0


### then redo the analysis above


In [122]:
my_field = "inc_grants"
g_in_2010v2 = df2[(df2["grant_type"] == "G") & (df2["year"] == 2010)].sort_values(
    my_field, ascending=False
)


num1 = g_in_2010v2.iloc[0]
num2 = g_in_2010v2.iloc[1]
total = g_in_2010v2[my_field].sum()
rest_total = total - num1[my_field] - num2[my_field]

print(
    f"The total of grants is  {total}\n"
    f" and the two highest values are:\n"
    f"         {num1['charity']:20s} with {num1[my_field]}\n"
    f"     and {num2['charity']:20s} with {num2[my_field]}\n"
    f"so the rest make up {rest_total}\n"
    f" which is {rest_total*100/num1[my_field]}%"
    f" of the value for {num1['charity']:20s}"
)

The total of grants is  428906688.0
 and the two highest values are:
         British Red Cross    with 347600000.0
     and Cancer Research UK   with 46628000.0
so the rest make up 34678688.0
 which is 9.97660759493671% of the value for British Red Cross   


In [155]:
def p_percent(vals: pd.Series) -> float:
    """aggregation function that returns the p percent value.
    i.e. the uncertainty (as a percent) of the estimate that
    second highest respondent make of the highest value.
    Assuming there are n items in the series, they are first
    sorted in descending order then we calculate the value
    p = 100* sum (N-2 lowest values)/highest value
    in case of values are all 0, returns 100
    """
    # print(f'vals is of type{type(vals)} and shape {vals.shape}')
    sorted = vals.sort_values(ascending=False)
    sum = sorted.sum()
    # print(sum)
    # print(sorted)

    sub_total = sum - sorted.iloc[0] - sorted.iloc[1]
    p = 100 * sub_total / sorted.iloc[0] if sum > 0 else 100.0
    print(f"{sum} - {sorted.iloc[0]} - {sorted.iloc[1]}= {sub_total},   so p = {p}")

    return p

### check this gives us what we want to threshold on

### i.e. we flag cells as sensitive if the value reported is less than (say) p=10

working is sahown for eacgh cell - can be commented out or change to debug info

In [156]:
grant_table_pvals = pd.crosstab(
    df2["year"], df2["grant_type"], values=df2[my_field], aggfunc=p_percent
)
grant_table_pvals

428906688.0 - 347600000.0 - 46628000.0= 34678688.0,   so p = 9.97660759493671
0.0 - 0.0 - 0.0= 0.0,   so p = 100.0
504137088.0 - 173171008.0 - 165531008.0= 165435072.0,   so p = 95.53277647953634
46544000.0 - 36645000.0 - 8073000.0= 1826000.0,   so p = 4.982944467185155
127533696.0 - 52900000.0 - 44369000.0= 30264696.0,   so p = 57.211145557655954
7192804.0 - 960417.0 - 675401.0= 5556986.0,   so p = 578.6013783596084
532464736.0 - 179178000.0 - 171564992.0= 181721728.0,   so p = 101.41966536070277
128380000.0 - 39118000.0 - 30369000.0= 58893000.0,   so p = 150.5521754690935
171878704.0 - 60100000.0 - 57500000.0= 54278704.0,   so p = 90.31398336106489
7779685.0 - 982879.0 - 857674.0= 5939132.0,   so p = 604.2587134326809
480105472.0 - 171692992.0 - 150663008.0= 157749472.0,   so p = 91.87880656188926
134480000.0 - 42228000.0 - 31064000.0= 61188000.0,   so p = 144.899119067917
203357216.0 - 74500000.0 - 69100000.0= 59757216.0,   so p = 80.21102818791947
8728330.0 - 712067.0 - 669620.0= 7

grant_type,G,N,R,R/G
year,,,,
2010,9.976608,100.000000,95.532776,4.982944
2011,57.211146,578.601378,101.419665,150.552175
2012,90.313983,604.258713,91.878807,144.899119
2013,80.211028,1031.734795,81.007505,140.335699
2014,27.689766,1030.425070,61.406976,154.241663
2015,28.602688,729.054045,54.780401,157.819349
